<a href="https://colab.research.google.com/github/Apurva6697/VEngageai/blob/main/VEngageai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import sqlite3
from tabulate import tabulate

class PhoneBookDB:
    def __init__(self, db_name='phone_book.db'):
        self.connection = sqlite3.connect(db_name)
        self.cursor = self.connection.cursor()
        self.create_phone_records_table()

    def create_phone_records_table(self):
        query = '''
            CREATE TABLE IF NOT EXISTS phone_records (
                Name TEXT,
                email TEXT,
                "Phone 1" TEXT,
                "Phone 2" TEXT
            );
        '''
        self.cursor.execute(query)
        self.connection.commit()

    def read_records_from_file(self, file_path, file_format='csv'):
        try:
            if file_format == 'csv':
                df = pd.read_csv(file_path)
            elif file_format == 'json':
                df = pd.read_json(file_path)

            # Convert DataFrame to list of dictionaries
            records = df.to_dict(orient='records')
        except Exception as e:
            print(f"An error occurred while reading records: {e}")
            records = []

        return records

    def execute_query(self, query):
        try:
            self.cursor.execute(query)
            self.connection.commit()
        except Exception as e:
            print(f"An error occurred: {e}")

    def select_records(self, condition=''):
        query = f"SELECT * FROM phone_records {condition} LIMIT 10;"
        self.cursor.execute(query)
        records = self.cursor.fetchall()
        return records

phone_book_db = PhoneBookDB()

# Problem 1: Read phone book records from CSV file
phone_book_db.execute_query("DELETE FROM phone_records")  # Clear existing data
file_path = 'phone_records.csv'
phone_records = phone_book_db.read_records_from_file(file_path, file_format='csv')

# Convert list of dictionaries to DataFrame
df = pd.DataFrame(phone_records)

# Insert records into the database
for _, row in df.iterrows():
    insert_query = f'''
        INSERT INTO phone_records (Name, email, "Phone 1", "Phone 2")
        VALUES ('{row['Name']}', '{row['email']}', '{row['Phone 1']}', '{row['Phone 2']}');
    '''
    phone_book_db.execute_query(insert_query)
# Display the initial records using Pandas DataFrame
print("\nInitial records:")
print(tabulate(df, headers='keys', tablefmt='psql'))


Initial records:
+----+---------------------+---------------------------+------------+------------+
|    | Name                | email                     |    Phone 1 |    Phone 2 |
|----+---------------------+---------------------------+------------+------------|
|  0 | Ashlee Walker       | shelbygomez@example.com   |  915769922 | 8936064636 |
|  1 | Stephanie Lee       | kaustin@example.com       | 5080243255 | 9701342284 |
|  2 | Amy Garrison        | jennifer90@example.org    | 6605271762 | 6056240648 |
|  3 | Mr. Keith Strong MD | adamsingleton@example.net |  357116779 | 5267841163 |
|  4 | Michael Griffin     | graymelanie@example.com   | 8074445120 | 4948681748 |
|  5 | Joe Choi            | nlopez@example.net        | 8605128162 | 5039085473 |
|  6 | Michael Miller      | youngdaniel@example.net   | 2368085023 | 5929663147 |
|  7 | Ashley Pena         | davistimothy@example.com  | 8577998786 |   36713725 |
|  8 | Tony Benson         | tyler76@example.net       | 1151374137 |

In [ ]:
from tabulate import tabulate
# Convert the records to a Pandas DataFrame
df = pd.DataFrame(phone_book_db.select_records(), columns=['Name', 'email', 'Phone 1', 'Phone 2'])

# Display the initial records
print("\nInitial records:")
print(tabulate(df, headers='keys', tablefmt='psql'))

# Delete the row with name 'Tony'
df = df[df['Name'] != 'Tony']

# Insert a new row with name 'Test'
new_row = pd.DataFrame({'Name': ['Test'], 'email': ['test@test.xtyz'], 'Phone 1': ['1234456'], 'Phone 2': ['1233233']})
df = pd.concat([df, new_row], ignore_index=True)

# Display the updated records
print("\nUpdated records:")
print(tabulate(df, headers='keys', tablefmt='psql'))

# Update the SQLite database with the modified DataFrame
phone_book_db.execute_query("DELETE FROM phone_records")  # Clear existing data
df.to_sql('phone_records', con=phone_book_db.connection, index=False, if_exists='append')



Initial records:
+----+---------------------+---------------------------+------------+------------+
|    | Name                | email                     |    Phone 1 |    Phone 2 |
|----+---------------------+---------------------------+------------+------------|
|  0 | Ashlee Walker       | shelbygomez@example.com   |  915769922 | 8936064636 |
|  1 | Stephanie Lee       | kaustin@example.com       | 5080243255 | 9701342284 |
|  2 | Amy Garrison        | jennifer90@example.org    | 6605271762 | 6056240648 |
|  3 | Mr. Keith Strong MD | adamsingleton@example.net |  357116779 | 5267841163 |
|  4 | Michael Griffin     | graymelanie@example.com   | 8074445120 | 4948681748 |
|  5 | Joe Choi            | nlopez@example.net        | 8605128162 | 5039085473 |
|  6 | Michael Miller      | youngdaniel@example.net   | 2368085023 | 5929663147 |
|  7 | Ashley Pena         | davistimothy@example.com  | 8577998786 |   36713725 |
|  8 | Tony Benson         | tyler76@example.net       | 1151374137 |

11